In [ ]:
def categorical_explanitory_binary_response_chart(chart_df, field, target, toggle=False, ):
    # the toggle switches which class is the "baseline" of the line chart
    drop_index = 2
    if toggle:
        drop_index = 1
    
    field_count  = chart_df[field].value_counts()
    field_count_df = field_count.to_frame()
    field_count_df.columns = ['count']
    
    field_target_pct = pd.crosstab(chart_df[field], chart_df[target], normalize='index') * 100
    field_target_pct = field_target_pct.reset_index()
    field_target_pct = field_target_pct.drop(field_target_pct.columns[-drop_index],axis=1)
    
    field_target_data = field_target_pct.merge(field_count_df, right_index=True, left_on=field).sort_values('count', ascending=False).reset_index(drop=True)
    
    print(field_target_data)
    
    fig, ax = plt.subplots(figsize=(10, 6))
    ax.set_xlabel(field)
    ax = sns.barplot(field_target_data[field], field_target_data['count'], alpha=0.8)
    ax.set_xticklabels(ax.get_xticklabels(), rotation=45, horizontalalignment='right')
    ax.set_ylabel('count (bars)')
    ax2 = ax.twinx()
    ax2 = sns.pointplot(x=field_target_data[field], y=field_target_data.iloc[:, -2], color='black', legend=False)
    ax2.set_ylabel('% {t} (line)'.format(t = target))
    plt.show()

In [ ]:
## just plot all the categoricals to explore and do one at a time later
feat_count = len(categorical_features)
plt.figure(figsize=(10, 6*feat_count))
for feat in categorical_features:
    categorical_explanitory_binary_response_chart(df, feat, 'income_50k')

In [ ]:
df_cont = df[continuous_features]
sns.heatmap(df_cont.corr(), annot = True)

In [ ]:
ax = scatter_matrix(df, figsize = (15,10))

In [ ]:
plt.figure(figsize=(12, 8))
sns.scatterplot(data=df, x="capital_losses", y="age", alpha=.6, hue=df['income_50k'].tolist())

In [ ]:
## Overlapping densities
plt.figure(figsize=(10, 8))
sns.distplot(df.loc[df['income_50k'] == ' - 50000.']['age'])
sns.distplot(df.loc[df['income_50k'] != ' - 50000.']['age'])

In [ ]:
#Violin Plots age
f, ax = plt.subplots(figsize = (15,8))

sns.violinplot(x= 'class_worker', y = 'age', hue = 'income_50k', data = df,
               split = True, inner = 'quart')
ax.set_xticklabels(ax.get_xticklabels(), rotation=45, horizontalalignment='right')

In [ ]:
from sklearn.decomposition import PCA
import matplotlib.pyplot as plt

def plot_pca(X, output_components=1):
    ## Perform PCA on the data to reduce the number of initial features 
    ## and to remove correlations that are common between pixel features 
    pca = PCA(n_components=X.shape[1])
    pca.fit(X)

    ## Inspect the explained variances to determine how many components to use  
    plt.subplots(figsize=(8, 8))
    # use n_components series to make x axis start at 1
    n_components = pd.Series(range(1,len(np.cumsum(pca.explained_variance_ratio_))+1))
    plt.plot(n_components, np.cumsum(pca.explained_variance_ratio_))
    plt.xlabel('Number of Components')
    plt.ylabel('Cumulative Explained Variance');
    
    ## Output the explained variances at output_components # of components
    output_str = 'Cumulative Explained variance at {n} components:'.format(n=output_components)
    print(output_str,sum(pca.explained_variance_ratio_[0:output_components]) )
    
plot_pca(df[continuous_features], 2)